## Problem Set 3

**Total points**: 24

**Background on two data sets**: here, we're going to use two datasets to practice regex patterns and merging/matching. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers, with more details on the policy background available at last spring's course page under the "Social impact practicum context" header: https://rebeccajohnson88.github.io/qss20/docs/sip_finalproject.html


The following datasets are located in `pset3_inputdata` (need to unzip): 

- `jobs`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


In [5]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re

import recordlinkage


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Regex and exact matching (10 points)



# 1.1 Load data on debarments and job postings (0 points)

Load the following datasets stored in `pset3_inputdata`
    
- Historical H2A debarments (debar.csv); call this `debar`
- Q1 2021 H2A job postings (jobs.csv); call this `jobs`


In [181]:
debar = pd.read_csv("/Users/S/Desktop/QSS 20/pset3/pset3_inputdata/debar.csv")
jobs = pd.read_csv("/Users/S/Desktop/QSS 20/pset3/pset3_inputdata/jobs.csv")


debar.head()
jobs.head()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


### 1.2 Try exact merge on business name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, subset to the following columns and print the rows with exact matches:

-`Employer_NAME` and `Name` 
- Date range of debarment (`Start date` and `End date` in `debar`)
- Location from each data (`City, State` in `debar` and `EMPLOYER_CITY` and `EMPLOYER_STATE` in `jobs`)

**Concepts tested and resources**: part A tests using `pd.merge` for exact merging; part B tests column subsetting 
   - Slides on exact merging: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/w22_slides/05_qss20_w22_unit5_mergingexact.pdf 
   - Code with examples of exact merging: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/03_merging_exact_solutions.ipynb 
   - Code with example of column subsetting: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/00_pandas_datacleaning_solutions.ipynb


In [7]:
jobs_debar = pd.merge(jobs, debar, how="inner", 
                      left_on = "EMPLOYER_NAME", right_on = "Name")

# jobs_debar[['EMPLOYER_NAME', 'Name']]
# jobs_debar[['EMPLOYER_NAME', 'Name', 'Start date', 'End date']]

jobs_debar[['Name','EMPLOYER_NAME', 'Start date', 'End date','City, State', 'EMPLOYER_CITY', 'EMPLOYER_STATE']]

,Name,EMPLOYER_NAME,Start date,End date,"City, State",EMPLOYER_CITY,EMPLOYER_STATE
0,Rafael Barajas,Rafael Barajas,9/23/2016,9/22/2017,"Sebring, Florida",Port St. Lucie,FL


## 1.3 Targeted regex

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 1.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 

**Resources**:
    - Activity code with list comprehension: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/00_pandas_datacleaning_solutions.ipynb 
    - Sampling from a list without replacement using the `random` module: https://note.nkmk.me/en/python-random-choice-sample-choices/ 
    

In [8]:
jobs['EMPLOYER_NAME'] = [x.upper() for x in jobs['EMPLOYER_NAME']]
debar['Name'] = [x.upper() for x in debar['Name']]

jobs.EMPLOYER_NAME.sample(n=15)
debar.Name.sample(n=15)

2225                   RIVERVIEW, LLP 
1562               C & C FONTENOT FARM
685            RICELAND PROPERTIES LLC
1913                 NEW HORIZON DAIRY
1330                      T CLARK LLC.
202                        LORI HOHERZ
317                           KLC FARM
185        THORNHILL RANCH PARTNERSHIP
1259                 HI-TECH FARMS INC
351                RP7 ENTERPRISE, LLC
923                BONNIE PLANTS, INC.
1162          AMERICAN RIVER COMPANIES
2468    SOUTHWING CRAWFISH COMPANY LLC
958          WESTERN RANGE ASSOCIATION
1021                 ALAN HEBERT FARMS
Name: EMPLOYER_NAME, dtype: object

100                   EVERGREEN PRODUCE
17     MAPLE RIDGE CUSTOM SERVICES, LLC
63                       RAFAEL BARAJAS
64                   GONZALO FERNANDEZ*
13                        REIMER'S INC.
65                          DONNA LUCIO
83                    CIRA CORTEZ LOPEZ
2                         TRUST NURSERY
105                         J & L FARMS
12          ROLLO FARM LABOR CONTRACTOR
84                 LANDMARK LANDSCAPING
53                        AGECY II, LLC
30              ANNABELLA LAND & CATTLE
67                       ROSALVA GARCIA
93                     DEAL FAMILY FARM
Name: Name, dtype: object

### 1.3.2 Cleaning up punctuation (6 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content

**Resources**:
    - Regex slides: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/w22_slides/06_qss20_w22_unit6_regex.pdf 
    - Regex activity code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/04_basicregex_solutions.ipynb

In [9]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [65]:
pattern = r'(INC|LLC|CO)\.'

npos_example_1 = re.sub(pattern, r'\1', pos_example_1)
npos_example_1
npos_example_2 = re.sub(pattern, r'\1', pos_example_2)
npos_example_2
nneg_example = re.sub(pattern, r'\1', neg_example)
nneg_example

'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

C. Use that pattern in conjunction with `re.sub` and list comprehension to clean the columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables

**Concepts and resources**: for the last part of row subsetting and printing two columns in each dataframe, use the `.loc` examples shown here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/00_pandas_datacleaning_solutions.ipynb


In [66]:
job_new = [re.sub(pattern, r'\1', EMPLOYER_NAME) for EMPLOYER_NAME in jobs.EMPLOYER_NAME]
debar_new = [re.sub(pattern, r'\1', Name) for Name in debar.Name]
jobs['name_clean'] = job_new
debar['name_clean'] = debar_new

In [67]:
jobs.loc[jobs.EMPLOYER_NAME != jobs.name_clean, ['name_clean','EMPLOYER_NAME']].head()
debar.loc[debar.Name != debar.name_clean, ['name_clean','Name']].head()

,name_clean,EMPLOYER_NAME
4,"DUNSON HARVESTING, INC","DUNSON HARVESTING, INC."
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC","FARM LABOR ASSOCIATION FOR GROWERS, INC."
14,"MCLAIN FARMS, INC","MCLAIN FARMS, INC."
17,"BONNIE PLANTS, INC","BONNIE PLANTS, INC."
18,"B & W QUALITY GROWERS, INC","B & W QUALITY GROWERS, INC."


,name_clean,Name
3,ANTON FERTILIZER INC,ANTON FERTILIZER INC.
4,"GREAT PLAINS FLUID SERVICE, INC","GREAT PLAINS FLUID SERVICE, INC."
5,PROMAX INC,PROMAX INC.
13,REIMER'S INC,REIMER'S INC.
19,CISCO PRODUCE INC,CISCO PRODUCE INC.


### Optional extra credit (1.3.3) regex to separate companies from individuals (4 points)

You notice some employers in `debar` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [192]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

pattern = r'(.*)(\s\(COMPANY\)\s)(AND\s)(.*)(\s\(INDIVIDUAL\))(.*)'
example1 = re.findall(pattern, pos_example)
example1
example2 = re.findall(pattern, neg_example)
example2

[('COUNTY FAIR FARM',
  ' (COMPANY) ',
  'AND ',
  'ANDREW WILLIAMSON',
  ' (INDIVIDUAL)',
  '*')]

[]

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [178]:
## your code here to create these columns


   
D. Print three columns for the rows in `debar` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [153]:
# your code here to print these columns

# 2 Fuzzy matching to match debarments to jobs (14 points)


## 2.1 Preprocessing state names (2 points)

You want to block on state but notice that states in `debar` have a mix of two digit codes and full state names (e.g., GA versus Georgia) while states in `jobs` are all two-digit state codes

A. Run the code below to load the `states` crosswalk (this matches state abbreviations with their full name). 

B. Use that crosswalk to create a field in `debar` that has the two-digit state abbreviation for all locations (hint: you may need to first split the `City, State` string on the ", " or use str.replace to extract the state into a new column before converting it to all abbreviations)

**Hint**: the GitHub issue contains the value_counts() for the cleaned two-digit states after this step

C. Use an `assert` statement to check that `all` the states in `debar` are two-digits after the cleaning

**Notes**: you can filter out states that are NaN

**Concepts and resources**:

- For part B, draw on regex or pandas str functions to create a `State` variable from the `City, State` column in `debar`
- Then, one approach is to use `pd.merge` to merge the state crosswalk onto debar (example head() shown in the GitHub issue)
- You may want to structure the `pd.merge` to only merge on rows *without* two-digit names to the crosswalk with those two-digit names and `pd.concat` to bind the dataframe back together
- How to structure an assert statement: https://www.w3schools.com/python/ref_keyword_assert.asp

In [62]:
## code to load state crosswalk
## source- https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c
cw_location = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/'
cw_filename = 'state_codes.html'
states = pd.read_html(cw_location + cw_filename)[0]
states

,Code,Description
0,AA,Armed Forces (the) Americas
1,AB,Alberta
2,AE,Armed Forces Europe
3,AK,Alaska
4,AL,Alabama
...,...,...
71,WA,Washington
72,WI,Wisconsin
73,WV,West Virginia
74,WY,Wyoming


In [189]:
## your code here to add two-digit state codes


debar[['City','State']]= debar['City, State'].str.split(', ', expand=True)
debar=debar[~debar.State.isnull()].copy()
debar_correct_state = debar[debar['State'].astype(str).str.count('\w') == 2].copy()
debar_incorrect_state = debar[debar['State'].astype(str).str.count('\w') != 2].copy()

debar_corrected_state = pd.merge(debar_incorrect_state, states, how="inner", 
                      left_on = "State", right_on = "Description")


debar_corrected_state_2 = debar_corrected_state[['Name', 'City, State', 'Violation','Duration', 'Start date', 
                     'End date', 'City', 'Code']]
debar_corrected_state_2 = debar_corrected_state_2.rename(columns={'Code': 'State'})

debar_combined = pd.concat([debar_correct_state, debar_corrected_state_2]).reset_index()

debar_combined.State.value_counts()
debar_corrected_state.head()
# debar_combined['State'].astype(str).str.count('\w')


assert (debar_combined['State'].astype(str).str.count('\w') ==2).all()


GA    19
TX    18
KS    10
FL    10
ND     6
UT     6
SD     5
KY     5
CA     4
AR     3
NY     3
MA     3
CO     3
LA     3
MT     2
ID     2
OK     2
MN     1
SC     1
AK     1
MD     1
IL     1
ME     1
VT     1
TN     1
NC     1
Name: State, dtype: int64

,Name,"City, State",Violation,Duration,Start date,End date,City,State,Code,Description
0,Reimer’s Inc.,"Lakin, Kansas",Impeding the Audit Process – Non- Response,2 years,8/23/2014,8/22/2016,Lakin,Kansas,KS,Kansas
1,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, Texas",Impeding the Audit Process – Partial- Response,2 years,7/20/2014,7/19/2016,Caddo,Texas,TX,Texas
2,John & Neta Leopky Farms,"Seminole, Texas",A violation of the requirements of § 655.135(j...,5 years,7/21/2016,7/20/2018,Seminole,Texas,TX,Texas
3,Triangle J Farms,"Seminole, Texas",Failure to pay or provide the required wages,2 years,7/21/2016,7/20/2018,Seminole,Texas,TX,Texas
4,AB Ranch,"Stephenville, Texas",Failure to respond to audit request,2 years,4/28/2017,4/27/2019,Stephenville,Texas,TX,Texas


## 2.2 step by step fuzzy matching (4 points)



A. Write fuzzy matching code (don't yet put inside a user-defined function, you'll do that in 2.3) that:

- Blocks on two-digit state code
- Finds matches based on similarity between the employer name (`name_clean`) in `debar` (uppercase and cleaned) and `name_clean` in `jobs` (uppercase and cleaned). You can choose which distance metric and threshold to use (feel free to set a threshold low enough to get some matches even if that leads to some false positives).

For the steps after compute, just take any match with non-zero value rather than using a classifier (so skip the k-means or e-m step from the class example code)

**Concepts and resources**:

- Solutions code here (more consolidated): https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/05_merging_fuzzy_activity_solutions.ipynb
- Example code here (more step by step): https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/w22_activities/solutions/05_merging_fuzzy_codeexample.ipynb 

**Hint**: you may need to deduplicate records in the datasets for the recordlinkage package to work. See drop_duplicates within pandas; the `subset` command within `drop_duplicates` allows you to only consider certain columns for duplicates; drop based on duplicates in the two-digit state code and `name_clean`. After you drop_duplicates, you may need to use reset_index to create a new index in the data, eg:

`dedup_df = df.drop_duplicates(subset = ['state', 'name_clean']).reset_index(drop = True)`

`dedup_df['df_ind'] = dedup_df.index`

In [147]:
## your fuzzy matching code here 
dedup_debar = debar_combined.drop_duplicates(subset = ['State', 'name_clean']).reset_index(drop = True)
dedup_debar['debar_ind'] = dedup_debar.index

jobs = jobs.rename(columns={'EMPLOYER_STATE': 'State'})
dedup_jobs = jobs.drop_duplicates(subset = ['State', 'name_clean']).reset_index(drop = True)
dedup_jobs['jobs_ind'] = dedup_jobs.index

indexer = recordlinkage.Index()
indexer.block("State")
candidate_links = indexer.index(dedup_debar, dedup_jobs)

thres = 0.85
c = recordlinkage.Compare()
c.string('name_clean', 'name_clean', 
               method='jarowinkler', threshold= thres)

compare_empname = c.compute(candidate_links, dedup_debar, dedup_jobs)

compare_empname_df = pd.DataFrame(compare_empname.reset_index())
compare_empname_df.columns = ["index_debar", "index_jobs", "name_match"]

match = compare_empname_df[compare_empname_df.name_match != 0]
match

<Index>

<Compare>

,index_debar,index_jobs,name_match
325,55,1154,1.0
694,67,1507,1.0
1013,18,361,1.0
1042,18,1710,1.0
5620,20,897,1.0
5762,66,838,1.0
7711,47,1011,1.0



B. Print the following columns in the resulting matched dataset and comment on examples of ones that seem like true positive matches and ones that seem like false positive matches:

- `name_clean` for jobs and debar
- `state` in debar
- `state` in jobs

In [149]:
dedup_debar['debar_index'] = dedup_debar.index
debar_add = pd.merge(match,
                   dedup_debar[['debar_index','name_clean','State']],
                   left_on = "index_debar",
                   right_on = "debar_index")


dedup_jobs['jobs_index'] = dedup_jobs.index
jobs_add = pd.merge(debar_add, 
                  dedup_jobs[['jobs_index', 'name_clean', 'State']],
                  left_on = "index_jobs",
                  right_on = "jobs_index",
                  suffixes = ["_debar", "_jobs"])

jobs_add


,index_debar,index_jobs,name_match,debar_index,name_clean_debar,State_debar,jobs_index,name_clean_jobs,State_jobs
0,55,1154,1.0,55,OLSON FARMS,ND,1154,NELSON FARMS,ND
1,67,1507,1.0,67,J & L FARMS,SD,1507,S & S FARMS,SD
2,18,361,1.0,18,F&W FARMS,KS,361,D&L FARMS,KS
3,18,1710,1.0,18,F&W FARMS,KS,1710,L&E FARMS,KS
4,20,897,1.0,20,SLASH E.V. RANCH LLP,CO,897,SLASH EV RANCH,CO
5,66,838,1.0,66,"LOV RANCH CO, LLLP DBA LOV RANCH",CO,838,LOV RANCH,CO
6,47,1011,1.0,47,DIXIE BELL #2,SC,1011,"DIXIE BELLE, INC",SC


Answer: Overall, we are pretty confident in the accuracy of our results. The ones that seem true to us are: SLASH E.V. RANCH, LOV RANCH CO, DIXIE BELL. The rest are the ones that we think are false positive.

## 2.3 Put the code from 2.2 into a fuzzy matching function that generalizes some steps (8 points)

You want to see how the matches change if you add the city and not only state as a field and also want to automate the process of matching a bit to try different distance thresholds.

A. Extract the City from the `City, State` column of `debar`

B. Convert that new `city` column to uppercase and convert the `EMPLOYER_CITY` column in `jobs` to uppercase

C. Write a function surrounding the code in `recordlinkage` that you wrote in problem 2.2 (so you don't need to recode the package from scratch) that (1) takes in each dataset, (2) blocks on two-digit state, and (3) fuzzy matches on employer name and employer city. See below notes on partial versus full credit for function structure. 

D. Execute the function with a couple different string distance thresholds and print the resulting matches for each

6 out of 8 points: function takes arguments for input datasets, varname to block on, two varnames to fuzzy match on (`name_clean` and `city)`, string distance function, and string distance threshold
    
8 out of 8: above but function is also general enough that it takes a variable # of strings to match on--- so should work if you either execute just using employer name or also work if you execute using employer name and employer city as the fuzzy variables.

**Concepts and resources**: same as above. One hint is that for us, the easiest way to get to the full credit solution was to feed the function a dictionary where each key corresponds to one string variable to fuzzy match on; the values contain that variable's name in each dataset, the distance metric, and the distance threshold.

In [180]:
debar['City'] = [x.upper() for x in debar['City']]
# jobs['EMPLOYER_NAME'] = [x.upper() for x in jobs['EMPLOYER_NAME']]

dedup_debar['city'] = [x.upper() for x in dedup_debar['City']]
dedup_jobs['city'] = [x.upper() for x in dedup_jobs['EMPLOYER_CITY']]

def fuzzymatch(df1, df2, varblock, dictionary):
    indexer = recordlinkage.Index()
    indexer.block(varblock)
    candidate_links = indexer.index(df1, df2)
    
    c = recordlinkage.Compare()
    for key, value in dictionary.items():
        c.string(dictionary[key][0], dictionary[key][1], method = dictionary[key][2], 
                 threshold = dictionary[key][3])
    
    compare_empname = c.compute(candidate_links, df1, df2)
    compare_empname_df = pd.DataFrame(compare_empname.reset_index())
    compare_empname_df.columns = ["index_df1", "index_df2", 'match_score1', 'match_score2']

    match = compare_empname_df[(compare_empname_df.match_score1 != 0) & (compare_empname_df.match_score2 != 0)]
    
    df1['index4match1'] = df1.index
    df1_add = pd.merge(match, df1,
                   left_on = "index_df1",
                   right_on = "index4match1")
    
    df2['index4match2'] = df2.index
    full = pd.merge(df1_add, df2,
                  left_on = "index_df2",
                  right_on = "index4match2",
                  suffixes = ["_df1", "_df2"])

    return full

check6_dictionary = {'matchvar1': ['name_clean', 'name_clean', 'jarowinkler', 0.65],
                     'matchvar2': ['city', 'city', 'jarowinkler', 0.65]}
check6 = fuzzymatch(dedup_debar, dedup_jobs, 'State', check6_dictionary)
check6[[col for col in check6.columns if "name_clean" in col or "city" in col or "_df" in col or "match_score" in col or "_index" in col]].head(10)

check7_dictionary = {'matchvar1': ['name_clean', 'name_clean', 'jarowinkler', 0.75],
                     'matchvar2': ['city', 'city', 'jarowinkler', 0.75]}
check7 = fuzzymatch(dedup_debar, dedup_jobs, 'State', check7_dictionary)
check7[[col for col in check7.columns if "name_clean" in col or "city" in col or "_df" in col or "match_score" in col or "_index" in col]]

check8_dictionary = {'matchvar1': ['name_clean', 'name_clean', 'jarowinkler', 0.85],
                     'matchvar2': ['city', 'city', 'jarowinkler', 0.85]}
check8 = fuzzymatch(dedup_debar, dedup_jobs, 'State', check8_dictionary)
check8[[col for col in check8.columns if "name_clean" in col or "city" in col or "_df" in col or "match_score" in col or "_index" in col]]



,index_df1,index_df2,match_score1,match_score2,State_df1,name_clean_df1,debar_index,city_df1,State_df2,name_clean_df2,jobs_index,city_df2
0,15,656,1.0,1.0,TX,SRT FARMS,15,MORTON,TX,TODD OTAHAL FARMS LLC,656,ROBSTOWN
1,15,2118,1.0,1.0,TX,SRT FARMS,15,MORTON,TX,BELDING FARMS LLC,2118,FORT STOCKTON
2,26,1610,1.0,1.0,TX,FIRST AMERICAN HOLDING,26,AUBREY,TX,GARRETT FLYING SERVICE INC,1610,DANBURY
3,65,1197,1.0,1.0,TX,DOVE CREEK FARMS,65,MOUNT VERNON,TX,MOORE'S HONEY FARM,1197,KOUNTZE
4,24,104,1.0,1.0,CA,YOLANDA CHAVEZ FARMING,24,SANTA MARIA,CA,"ALCO HARVESTING, LLC",104,SANTA MARIA
5,24,504,1.0,1.0,CA,YOLANDA CHAVEZ FARMING,24,SANTA MARIA,CA,CHASEN GOAT GRAZING,504,SANTA ROSA
6,62,174,1.0,1.0,FL,RUBEN RUIZ (DESOTO HARVESTING),62,ARCADIA,FL,"SUN CITRUS HARVESTING, INC",174,ARCADIA
7,62,202,1.0,1.0,FL,RUBEN RUIZ (DESOTO HARVESTING),62,ARCADIA,FL,"CISNEROS HARVESTING, INC",202,ARCADIA
8,42,1518,1.0,1.0,GA,CITY PINESTRAW AND HARVESTING,42,BAXLEY,GA,"NUNEZ PINE STRAW, INC",1518,BAXLEY
9,51,154,1.0,1.0,GA,MYRKA MIREYA CARDENAS,51,LYONS,GA,MARTIN GRANADOS,154,COLLINS


,index_df1,index_df2,match_score1,match_score2,State_df1,name_clean_df1,debar_index,city_df1,State_df2,name_clean_df2,jobs_index,city_df2
0,60,103,1.0,1.0,GA,DAVID C. MARTINEZ,60,LYONS,GA,JOSE H. MARTINEZ,103,LYONS
1,20,897,1.0,1.0,CO,SLASH E.V. RANCH LLP,20,RIFLE,CO,SLASH EV RANCH,897,RIFLE
2,43,838,1.0,1.0,CO,E.V. RANCH LLP,43,RIFLE,CO,LOV RANCH,838,RIFLE
3,66,838,1.0,1.0,CO,"LOV RANCH CO, LLLP DBA LOV RANCH",66,RIFLE,CO,LOV RANCH,838,RIFLE


,index_df1,index_df2,match_score1,match_score2,State_df1,name_clean_df1,debar_index,city_df1,State_df2,name_clean_df2,jobs_index,city_df2
0,20,897,1.0,1.0,CO,SLASH E.V. RANCH LLP,20,RIFLE,CO,SLASH EV RANCH,897,RIFLE
1,66,838,1.0,1.0,CO,"LOV RANCH CO, LLLP DBA LOV RANCH",66,RIFLE,CO,LOV RANCH,838,RIFLE


# 3. Optional extra credit (up to 6 points)

- For 2 points extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 6 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 

In [21]:
## your code here